In [1]:
import pandas as pd
import os
import sys
import fbprophet as fb
from datetime import datetime as dt

In [2]:
import numpy as np
import csv

In [3]:
dim_cols_dict= {
                'item':{'key':['id'],
						'incols':['id','item_id','dept_id','cat_id'],
						'outcols':['id','item_id','dept_id','cat_id']
						},
               'store':{'key':['store_id'],
			            'incols':['store_id','state_id'],
						'outcols': ['store_id','state_id'] 
						}
              }

In [4]:
def process_dimension(file_name,dim_cols_dict,dim_type):
    dim_name = 'dim_' + dim_type
    dim_key_cols = dim_cols_dict[dim_type]['key']
    dim_in_all_cols = dim_cols_dict[dim_type]['incols']
    dim_out_cols = dim_cols_dict[dim_type]['outcols']  
    file_path = r'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\\'
    #file_name = 'sales_train_validation.csv'
    dim_sk = dim_type + '_sk'
    dim_max_sk_csv = dim_name + '_max_sk.csv'
    dim_idx_lkp_csv = dim_name + '_idx_lkp.csv'
    dim_csv = dim_name + '.csv'
    dim_hash = dim_type + '_hash_key'
    dim_out_all_cols = list(dim_out_cols)
    dim_idx_cols = list(dim_key_cols)
    dim_out_all_cols.extend([dim_sk,dim_hash,'rec_hash'])
    dim_idx_cols.append(dim_sk)
  
    df1 = pd.read_csv(file_path + file_name )
    df1 = df1[dim_in_all_cols]
    df1.drop_duplicates(inplace=True)
    df1.reset_index(drop=True,inplace=True)
    try:
        df_max_lkp = pd.read_csv(file_path + dim_max_sk_csv)
        max_sk = df_max_lkp.iat[0,0]
    except:
        max_sk = 1
    try:
        df_dim_idx_lkp = pd.read_csv(file_path + dim_idx_lkp_csv,index_col=dim_hash)
    except:
        df_dim_idx_lkp = pd.DataFrame()
  
   
    def lookup_fun(x):
        try:
            return(df_dim_idx_lkp.at[x,dim_sk])
        except:
            return(-1)
    df1[dim_hash] = pd.util.hash_pandas_object(df1[dim_key_cols],index=False)
    df1['rec_hash'] = pd.util.hash_pandas_object(df1[dim_out_cols],index=False)
    df1[dim_sk] = df1[dim_hash].apply(lambda x: lookup_fun(x))
    df1_dim_updates = df1[df1[dim_sk] != -1]
    df1_dim_updates.reset_index(drop=True,inplace=True)
    df1_dim_inserts = df1[df1[dim_sk] == -1]
    if len(df1_dim_inserts) > 0:
        df1_dim_inserts.reset_index(drop=True,inplace=True)
        df1_dim_inserts[dim_sk] = df1_dim_inserts.index + max_sk
        max_sk = df1_dim_inserts[dim_sk].max() + 1
        if max_sk == max_sk:
            pd.DataFrame.from_dict({'max_sk':[max_sk]}).to_csv(file_path + dim_max_sk_csv,index=None)
            df1_dim_inserts.set_index(dim_hash,inplace=True)
            df_dim_idx_lkp = df_dim_idx_lkp.append(df1_dim_inserts[dim_idx_cols])
            df_dim_idx_lkp.to_csv(file_path + dim_idx_lkp_csv,header=True,index=True)
            df1_dim_inserts.reset_index(inplace=True)
        try:
            df1_dim_current = pd.read_csv(file_path + dim_csv)
        except:
            df1_dim_current = pd.DataFrame(columns=dim_out_all_cols)
        if len(df1_dim_updates) > 0:
            df1_dim_updates['check_dup_up'] = df1_dim_updates['rec_hash'].isin(df1_dim_current['rec_hash'])
            df1_dim_updates = df1_dim_updates[df1_dim_updates.check_dup_up == False]
            df1_dim_current['check_up'] = df1_dim_current['rec_hash'].isin(df1_dim_updates['rec_hash'])
            df1_dim_current = df1_dim_current[df1_dim_current.check_up == False]
            df1_dim_current = df1_dim_current[dim_out_all_cols].append(df1_dim_updates[dim_out_all_cols])
        df1_dim_current = df1_dim_current[dim_out_all_cols].append(df1_dim_inserts[dim_out_all_cols])
        df1_dim_current.to_csv(file_path + dim_csv ,header=True,index=None)
        return

In [5]:
file_name = 'sales_train_evaluation.csv'

In [6]:
process_dimension(file_name,dim_cols_dict,'item')

In [7]:
def read_dim_for_fact(dim_cols_dict,dim_type):
  dim_name = 'dim_' + dim_type
  dim_key_cols = dim_cols_dict[dim_type]['key']
  dim_in_all_cols = dim_cols_dict[dim_type]['incols']
  dim_out_cols = dim_cols_dict[dim_type]['outcols']  
  file_path = r'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\\'
  #file_name = 'sales_train_validation.csv'
  dim_sk = dim_type + '_sk'
  dim_max_sk_csv = dim_name + '_max_sk.csv'
  dim_idx_lkp_csv = dim_name + '_idx_lkp.csv'
  dim_csv = dim_name + '.csv'
  dim_hash = dim_type + '_hash_key'
  dim_out_all_cols = list(dim_out_cols)
  dim_idx_cols = list(dim_key_cols)
  dim_out_all_cols.extend([dim_sk,dim_hash,'rec_hash'])
  dim_idx_cols.append(dim_sk)
  df_dim_idx_lkp = pd.read_csv(file_path + dim_idx_lkp_csv,index_col=dim_hash)
  return(df_dim_idx_lkp)

In [8]:
def add_sk_to_fact(df_fact,dim_cols_dict,dim_type):
  dim_hash = dim_type + '_hash_key'
  dim_key_cols = dim_cols_dict[dim_type]['key']
  dim_in_all_cols = dim_cols_dict[dim_type]['incols']
  dim_in_all_cols.append(dim_hash)
  dim_sk = dim_type + '_sk'
  if dim_sk not in df_fact.columns:
    df_fact[dim_hash] = pd.util.hash_pandas_object(df_fact[dim_key_cols],index=False)
    df_dim_idx_lkp = read_dim_for_fact(dim_cols_dict,dim_type)
    def lookup_fun(x):
      try:
        return(df_dim_idx_lkp.at[x,dim_sk])
      except:
        return(-1)
    df_fact.insert(1,dim_sk,df_fact[dim_hash].apply(lambda x:lookup_fun(x)))
    #df_fact[dim_sk] = df_fact[dim_hash].apply(lambda x:lookup_fun(x))
    df_dim_placeholder = df_fact[dim_key_cols][df_fact[dim_sk] == -1]
    df_fact.drop(columns=dim_in_all_cols,inplace=True)
    return(df_fact,df_dim_placeholder)
  else:
    return(df_fact,pd.DataFrame())

In [40]:
file_path = 'C:\\Users\\Rohan\\Desktop\\Kaggle\\M5_Walmart\\'
in_file = 'sales_train_evaluation_10_items.csv'
normalized_file = 'sales_train_evaluation_normalized_test4.csv'

In [41]:
read_fact_input = pd.read_csv(file_path + in_file)

In [42]:
read_fact_input,df_item_placeholder = add_sk_to_fact(read_fact_input,dim_cols_dict,'item')

In [45]:
read_fact_input.head()

,item_sk,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,1,CA,0,0,0,0,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,2,CA,0,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,3,CA,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,4,CA,0,0,0,0,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,5,CA,0,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [44]:
read_fact_input.drop(columns=['store_id'],inplace=True)

In [48]:
read_fact_input.to_csv(file_path + 'sales_evaluate_mapped_sk.csv',index=False)

In [51]:
in_file2 = 'sales_evaluate_mapped_sk.csv'

In [53]:
file_open = open(file_path + in_file2)
file_contents = csv.reader(file_open)
file_write = open(file_path + normalized_file,'a',newline='')
print(F"start time is {dt.now()}")
dim_fields = 2
transaction_id = 1
for i,row in enumerate(file_contents):
    start = 2
    if i == 0:
        top_row = row
        normalize = row[:dim_fields]
        normalize.extend(['date','unit_sold','trasaction_id'])
        row1 = normalize
        row1_final = [row1[0]]
        row1_final.extend(row1[-4:-1])
        row_write = csv.writer(file_write)
        row_write.writerow(row1_final)
        nu_of_fields = len(top_row) - dim_fields
    else:
        start = dim_fields
        for j in range(1,nu_of_fields+1):
            normalize_data = row[:dim_fields]
            #print(j,nu_of_fields,start)
            date = top_row[start]
            #print(date)
            next_trans = row[start]
            normalize_data.extend([date,next_trans,transaction_id])
            #print(normalize_data)
            transaction_id = transaction_id + 1
            start = start + 1
            row2 = normalize_data
            #print(row2)
            row2_final = [row2[0]]
            row2_final.extend(row2[-4:-1])
            row_write.writerow(row2_final)
file_write.close()
print(F"end time is {dt.now()}")

start time is 2020-06-24 07:48:49.743428
end time is 2020-06-24 07:48:49.823406


In [54]:
df_calendar = pd.read_csv(file_path + 'calendar.csv',parse_dates=[0])
df_calendar[['unwanted','date_sk']] = df_calendar.date_key.str.split("_",expand=True)
df_calendar.drop(columns=['unwanted'],inplace=True)
df_calendar.year = df_calendar.year.astype('int64')
df_calendar.date_sk = df_calendar.date_sk.astype('int64')

In [55]:
df_read_full_file = pd.read_csv(file_path + normalized_file)

In [57]:
df_read_full_file.head()

,item_sk,state_id,date,unit_sold
0,1,CA,d_1,0
1,1,CA,d_2,0
2,1,CA,d_3,0
3,1,CA,d_4,0
4,1,CA,d_5,0


In [58]:
df_item_id = df_read_full_file[['item_sk','state_id']][df_read_full_file.duplicated(subset=['item_sk','state_id']) == False]

In [59]:
df_item_id.reset_index(inplace=True)

In [60]:
def create_holiday(df_cal,state):      
        if state == 'CA':
            df_holiday = df_cal[(df_cal.event_name_1.notna()) | (df_cal.snap_CA == 1)]
        elif state == 'TX':
            df_holiday = df_cal[(df_cal.event_name_1.notna()) | (df_cal.snap_TX == 1)]
        else:
            df_holiday = df_cal[(df_cal.event_name_1.notna()) | (df_cal.snap_WI == 1)]
        df_holiday['holiday'] = np.where(df_holiday.event_name_1.isna(),'snap',df_holiday.event_name_1)
        df_holiday.rename(columns={'date':'ds'},inplace=True)
        return(df_holiday[['holiday','ds']])

In [61]:
states = []
for i in df_calendar.columns:
    if 'snap' in i.lower():
        states.append(i.split('_')[1])

In [62]:
holiday_ca = create_holiday(df_calendar,states[0])
holiday_tx = create_holiday(df_calendar,states[1])
holiday_wi = create_holiday(df_calendar,states[2])

C:\Users\Rohan\miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [63]:
def calendar_lookup(x):
    return(df_calendar.at[x,'date_key'])

In [65]:
for index,i in df_item_id.iterrows():
    if index >= 0 and index < 100:
        print(F"loop start time {datetime.strftime(datetime.now(),format='%Y%m%d %H:%M:%S')}")
        print(i['item_sk'])
        df_item_sk_1 = df_read_full_file[(df_read_full_file.item_sk == i['item_sk'])]
        df_item_sk_1[['unwanted','date_sk2']] = df_item_sk_1.date.str.split("_",expand=True)
        df_item_sk_1.drop(columns=['unwanted','date'],inplace=True)
        df_item_sk_1.rename(columns={'date_sk2':'date_sk'},inplace=True)
        df_item_sk_1.date_sk = df_item_sk_1.date_sk.astype('int64')
        df_item_sk_1 = df_item_sk_1.join(df_calendar.set_index('date_sk'),on=['date_sk'],rsuffix='xxx')
        df1_for_fb = df_item_sk_1[['date','unit_sold']]
        df1_for_fb.rename(columns={'date':'ds','unit_sold':'y'},inplace=True)
        if i['state_id'] == 'CA':
            m = fb.Prophet(holidays=holiday_ca,daily_seasonality=True)
            print("using cali holiday calendar")
        elif i['state_id'] == 'TX':
            m = fb.Prophet(holidays=holiday_tx,daily_seasonality=True)
            print("using texas holiday calendar")
        else:
            m = fb.Prophet(holidays=holiday_wi,daily_seasonality=True)
            print("using wisconsin holiday calendar")
        m.fit(df1_for_fb)
        future = m.make_future_dataframe(periods=28)
        forecast = m.predict(future)
        forecast = forecast[['ds','yhat']][forecast.ds > df1_for_fb.ds.max()]
        df_calendar.set_index(['date'],inplace=True)
        forecast['date_key'] = forecast.ds.apply(lambda x: calendar_lookup(x))
        forecast = forecast[['date_key','yhat']]
        forecast.set_index(['date_key'],inplace=True)
        forecast = forecast.transpose()
        forecast = forecast.iloc[:,-28:]
        forecast.insert(0,'item_sk',i['item_sk'])
        #forecast.insert(1,'store_sk',i['store_sk'])
        df_calendar.reset_index(inplace=True)
        forecast.to_csv(file_path + 'walmart_prediction5.csv', index=False, header=False, mode='a')
        print(F"loop end time {datetime.strftime(datetime.now(),format='%Y%m%d %H:%M:%S')}")

loop start time 20200624 07:53:21
1
using cali holiday calendar


C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

loop end time 20200624 07:53:27
loop start time 20200624 07:53:27
2
using cali holiday calendar
loop end time 20200624 07:53:33
loop start time 20200624 07:53:33
3
using cali holiday calendar
loop end time 20200624 07:53:39
loop start time 20200624 07:53:39
4
using cali holiday calendar
loop end time 20200624 07:53:48
loop start time 20200624 07:53:48
5
using cali holiday calendar
loop end time 20200624 07:53:56
loop start time 20200624 07:53:56
6
using cali holiday calendar
loop end time 20200624 07:54:04
loop start time 20200624 07:54:04
7
using cali holiday calendar
loop end time 20200624 07:54:10
loop start time 20200624 07:54:10
8
using cali holiday calendar
loop end time 20200624 07:54:18
loop start time 20200624 07:54:18
9
using cali holiday calendar
loop end time 20200624 07:54:25


In [27]:
df_calendar.reset_index(inplace=True)

In [18]:
df_read_full_file.head()

,id,state_id,date,unit_sold
0,HOBBIES_1_001_CA_1_validation,CA,d_1,0
1,HOBBIES_1_001_CA_1_validation,CA,d_2,0
2,HOBBIES_1_001_CA_1_validation,CA,d_3,0
3,HOBBIES_1_001_CA_1_validation,CA,d_4,0
4,HOBBIES_1_001_CA_1_validation,CA,d_5,0


In [21]:
for index,i in df_item_id.iterrows():
    print(index,i)

0 id    HOBBIES_1_001_CA_1_validation
Name: 0, dtype: object
1913 id    HOBBIES_1_002_CA_1_validation
Name: 1913, dtype: object
3826 id    HOBBIES_1_003_CA_1_validation
Name: 3826, dtype: object
5739 id    HOBBIES_1_004_CA_1_validation
Name: 5739, dtype: object
7652 id    HOBBIES_1_005_CA_1_validation
Name: 7652, dtype: object
9565 id    HOBBIES_1_006_CA_1_validation
Name: 9565, dtype: object
11478 id    HOBBIES_1_007_CA_1_validation
Name: 11478, dtype: object
13391 id    HOBBIES_1_008_CA_1_validation
Name: 13391, dtype: object
15304 id    HOBBIES_1_009_CA_1_validation
Name: 15304, dtype: object
17217 id    HOBBIES_1_010_CA_1_validation
Name: 17217, dtype: object


In [24]:
df_item_id.head()

,id,state_id
0,HOBBIES_1_001_CA_1_validation,CA
1913,HOBBIES_1_002_CA_1_validation,CA
3826,HOBBIES_1_003_CA_1_validation,CA
5739,HOBBIES_1_004_CA_1_validation,CA
7652,HOBBIES_1_005_CA_1_validation,CA


In [32]:
df_item_id

,id,state_id
0,HOBBIES_1_001_CA_1_validation,CA
1913,HOBBIES_1_002_CA_1_validation,CA
3826,HOBBIES_1_003_CA_1_validation,CA
5739,HOBBIES_1_004_CA_1_validation,CA
7652,HOBBIES_1_005_CA_1_validation,CA
9565,HOBBIES_1_006_CA_1_validation,CA
11478,HOBBIES_1_007_CA_1_validation,CA
13391,HOBBIES_1_008_CA_1_validation,CA
15304,HOBBIES_1_009_CA_1_validation,CA
17217,HOBBIES_1_010_CA_1_validation,CA


In [37]:
for index,i in df_item_id.iterrows():
    #if index >= 0 and index < 100:
        print(F"loop start time {datetime.strftime(datetime.now(),format='%Y%m%d %H:%M:%S')}")
        print(index,i['id'])

loop start time 20200623 07:45:14
0 HOBBIES_1_001_CA_1_validation
loop start time 20200623 07:45:14
1 HOBBIES_1_002_CA_1_validation
loop start time 20200623 07:45:14
2 HOBBIES_1_003_CA_1_validation
loop start time 20200623 07:45:14
3 HOBBIES_1_004_CA_1_validation
loop start time 20200623 07:45:14
4 HOBBIES_1_005_CA_1_validation
loop start time 20200623 07:45:14
5 HOBBIES_1_006_CA_1_validation
loop start time 20200623 07:45:14
6 HOBBIES_1_007_CA_1_validation
loop start time 20200623 07:45:14
7 HOBBIES_1_008_CA_1_validation
loop start time 20200623 07:45:14
8 HOBBIES_1_009_CA_1_validation
loop start time 20200623 07:45:14
9 HOBBIES_1_010_CA_1_validation


In [68]:
row1 = ['id','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17','F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28']
file_write = open(file_path + 'walmart_prediction7.csv','w',newline='')
row_write = csv.writer(file_write)

row_write.writerow(row1)
file_write.close()